In [32]:
import warnings
warnings.filterwarnings("ignore")

In [33]:
import pandas as pd
test = pd.read_csv("/content/test_race.csv")
train = pd.read_csv("/content/train_race.csv")

In [34]:
train.isnull().sum(), test.isnull().sum() # 없다.

(workclass         0
 education         0
 marital.status    0
 occupation        0
 relationship      0
 race              0
 sex               0
 capital.gain      0
 capital.loss      0
 income            0
 dtype: int64,
 workclass         0
 education         0
 marital.status    0
 occupation        0
 relationship      0
 sex               0
 capital.gain      0
 capital.loss      0
 income            0
 dtype: int64)

In [35]:
train.head(2)

,workclass,education,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,income
0,Private,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,<=50K
1,Private,Some-college,Separated,Sales,Unmarried,White,Female,0,1617,<=50K


In [36]:
test.head(2) # race가 없다.

,workclass,education,marital.status,occupation,relationship,sex,capital.gain,capital.loss,income
0,Private,Some-college,Married-civ-spouse,Handlers-cleaners,Husband,Male,0,0,<=50K
1,Private,Some-college,Never-married,Sales,Own-child,Male,0,0,<=50K


In [37]:
train.shape, test.shape

((22792, 10), (9769, 9))

In [39]:
pd.set_option('display.max_columns', None)

In [20]:
# target 확인
train.race.value_counts()

White                 19457
Black                  2188
Asian-Pac-Islander      744
Amer-Indian-Eskimo      217
Other                   186
Name: race, dtype: int64

### 정답 데이터 나누기, id

In [40]:
target = train.pop('race')

### 데이터 나눠서 스케일링

In [42]:
from sklearn.preprocessing import RobustScaler, LabelEncoder
cols = train.select_dtypes(exclude='object').columns
rs = RobustScaler()
train[cols] = rs.fit_transform(train[cols])
test[cols] = rs.transform(test[cols])

cols = train.select_dtypes(include = 'object').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train,
                                                  target,
                                                  test_size=0.2,
                                                  random_state=11)

### 모델 선정, 훈련, 예측

In [46]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=5, max_depth=7, random_state=11)
model.fit(X_train, y_train)
pred = model.predict(X_val)
pred

array(['White', 'White', 'White', ..., 'White', 'White', 'White'],
      dtype=object)

### 평가

In [47]:
from sklearn.metrics import f1_score
f1_score(y_val, pred, average='weighted')

0.7930583198235454

### 정답 제출

In [49]:
pred = model.predict(test)
result = pd.DataFrame({'pred' :pred})
result.pred.value_counts()

White                 9765
Black                    3
Asian-Pac-Islander       1
Name: pred, dtype: int64

In [50]:
result.to_csv('result.csv')